In [ ]:
!pip install torch torchvision torchaudio

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [58]:
tokens = tokenizer.encode('It was pretty decent. Really nice', return_tensors='pt')

In [60]:
tokenizer.decode(tokens[0])

'[CLS] it was pretty decent. really nice [SEP]'

In [62]:
result = model(tokens)

In [64]:
int(torch.argmax(result.logits)) + 1

4

In [88]:
r = requests.get('https://www.yelp.com/biz/fire-wok-arbutus-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all("p", {'class':regex})
reviews = [result.text for result in results]

In [92]:
reviews[0]

"Got some takeout here recently. Overall, not bad!I started out with some of the fried pork dumplings, thanks to many other recs on here and they did not disappoint! Crispy, porky, with a delicious sauce that had the perfect balance of sweet, sour, and salty. Would get again.I also ordered the Orange Chicken. Nice crispy breading without being too much or too heavy. The pieces were also cut into manageable-sized chunks rather than some of those enormous pieces you sometimes get. The taste was fine. I only wish there was a little bit more broccoli and a little bit more sauce to match the amount of rice they gave me. Otherwise, very good.I had it delivered and their website estimated about 55 minutes until arrival, but the driver got there in about 30, so I hope that speedy service is typical! Prices are fair. I'd order from here again."

In [98]:
import numpy as np
import pandas as pd

In [100]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [106]:
df['review'].iloc[0]

"Got some takeout here recently. Overall, not bad!I started out with some of the fried pork dumplings, thanks to many other recs on here and they did not disappoint! Crispy, porky, with a delicious sauce that had the perfect balance of sweet, sour, and salty. Would get again.I also ordered the Orange Chicken. Nice crispy breading without being too much or too heavy. The pieces were also cut into manageable-sized chunks rather than some of those enormous pieces you sometimes get. The taste was fine. I only wish there was a little bit more broccoli and a little bit more sauce to match the amount of rice they gave me. Otherwise, very good.I had it delivered and their website estimated about 55 minutes until arrival, but the driver got there in about 30, so I hope that speedy service is typical! Prices are fair. I'd order from here again."

In [108]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [110]:
sentiment_score(df['review'].iloc[0])

4

In [112]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [116]:
df

,review,sentiment
0,"Got some takeout here recently. Overall, not b...",4
1,"Quite the disappointment, ordered Egg Drop Wo...",2
2,This place had surprisingly delicious Chinese ...,2
3,I don't know what's up with these reviews but ...,2
4,This is good place to go to pick up Chinese fo...,4
5,Very good carry out and they deliver. They jus...,4
6,Great food and service! This is really nice fa...,5
7,My son moved in nearby so after helping him mo...,3
8,"Ordered bourbon chicken, sesame chicken, and f...",5
9,My go to Chinese take out. Fire wok does an ex...,5


In [118]:
reviews[2]

"This place had surprisingly delicious Chinese food. Blown away.Chicken & BroccoliExcellent sauce, great flavors and perfect consistency. The chicken was tender and the broccoli crisp and tender, while not being mushy. Cheesesteak EggrollHere is where they fell short. There was a speck of steak inside the eggroll. It was more of a cheese eggroll than anything. Disappointed on this one. Fried pork Dumplings Excellent! Perfect sear on the outside, golden brown and soft in the areas that weren't seared. Delicious flavors, could really get the hints of ginger. Sauce it came with was the perfect amount of sweet and salty. #FeedsMySoul. Singapore Mei FunNoodles we cooked perfectly. Excellent amount of curry. All of the meat was tender and flavorful. Super impressed for this spot. Definitely check them out."